### Plot data from the resampled netCDF files and compare with Matlab processed data
To run this Notebook:
- Install with instructions at https://bitbucket.org/mbari/auv-python
- `cd auv-python`
- `poetry install && poetry shell`
- `cd notebooks && jupyter notebook 4.0-mpm-resampled-matlab.ipynb`

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'))
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
import pandas as pd
from logs2netcdfs import BASE_PATH, LOG_FILES

# Pick auv_name available from locally processed files
auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                               if f != '.DS_Store'])
display(auv_picker)

In [2]:
# Pick resampled netCDF file on local system
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(missions_dir))
                                                   if f != '.DS_Store'])
display(mission_picker)

In [3]:
from glob import glob
mission = mission_picker.value

# Get handle to corresponding Matlab generated netCDF on opendap server
year = mission.split('.')[0]
yd = mission.split('.')[1]
seq = mission.split('.')[2]
survey = f"{year}_{yd}_{seq}_{yd}_{seq}"    # Most surveys named like this
matlab_path = os.path.join('http://dods.mbari.org/opendap/data/auvctd/surveys/', 
                            mission.split('.')[0], 'netcdf', f"{auv_name}_{survey}_decim.nc")

# Pick from possible set of resampled netCDF files on local system
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission)
resampled_picker = widgets.Dropdown(options= [f for f in 
                                                glob(os.path.join(netcdfs_dir, f"{auv_name}_{mission}_*S.nc"))
                                                if f != '.DS_Store'],
                                    layout={'width': 'max-content'})
display(resampled_picker)

In [8]:
resampled_nc = resampled_picker.value
try:
    ds = xr.open_dataset(resampled_nc)
except (FileNotFoundError, OSError):
    print(f'{resampled_nc} not found')
try:
    ds_matlab = xr.open_dataset(matlab_path)
except (FileNotFoundError, OSError):
    print(f'{matlab_path} not found')

# Pick a variable from the resampled netCDF
var_picker = widgets.Dropdown(options=list(ds), layout={'width': 'max-content'})
display(var_picker)

In [9]:
# Pick a variable from the Matlab generated netCDF file
var_picker_matlab = widgets.Dropdown(options=list(ds_matlab), layout={'width': 'max-content'})
display(var_picker_matlab)

In [11]:
variable = var_picker.value
variable_matlab = var_picker_matlab.value
 
var = getattr(ds, variable)
try:
    d_var = getattr(ds_matlab, variable_matlab)
except AttributeError as e:
    d_var = xr.DataArray([])

# Plot first hour of locally generated variable with the chosen Matlab generated variable
first_hour = ds.time.values[0].astype('str').split(':')[0]    
df = pd.concat([ds[variable].sel(time=first_hour).to_pandas().rename(f"auv-python"),
                 ds_matlab[variable_matlab].sel(time=first_hour).to_pandas().rename(f"Matlab")], axis=1)
display(df.hvplot(ylabel=f"{var.attrs['long_name']} [{var.attrs.get('units', '')}]"))